In [3]:
import os
import pandas as pd
import scraper
import translator
#import sentimentAnalysis
#import graphAnalysis
import utils
import importlib
importlib.reload(utils)
importlib.reload(scraper)
from nltk.corpus import wordnet
from itertools import chain
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [6]:
#Wczytywanie danych - Data Load
oneonetwo = pd.read_csv("Suomi112_cpd.csv", encoding='utf-8-sig')
sos_live = pd.read_csv("SosLive_cpd.csv", encoding='utf-8-sig')
#Połączenie danych - Data Concat
data = pd.concat([oneonetwo,sos_live],ignore_index = True)

In [7]:
#Lista Indykatorów - List of indicators
lista_indykatorow = ["perceived_ease_of_use", "perceived_usefulness", "satisfaction", "attitude", "behavioral_intentions"]
#Stworzyłem poszczególne listy słów kluczy: - I created lists of the keyword:
slowa_klucze  = \
{
    "perceived_ease_of_use" : ["simple","easy","intuitive","convenience","explainable","quick","speed","children","elderly"],
    "perceived_usefulness"  : ["efficient", "useful", "helpful", "work", "effective","revolutionary","implement"],
    "satisfaction"          : ["satisfaction","satisfy", "fulfill", "gratify", "meet", "beneficial", "content", "happy", "appeasement"],
    "attitude"              : [ "important", "emotion", "attitude", "opinion", "grade"],
    "behavioral_intentions" : ["usage","intentions", "interaction","plan","utilize","behavioral"],
}

In [8]:
# Ta funkcja tworzy listę synonimóœ i nie tylku z listy słów - This function creates a list of synonyms and more from the list of words
def dobierz_synonimy_i_nie_tylko(slowa):
    slowa_klucze_zwrotne = []
    for slowo in slowa:
        slowa_klucze_zwrotne.append(slowo)
        #Synonims:
        synonimy = []
        for syn in wordnet.synsets(slowo):
            for s in syn.lemmas():
                synonimy.append(s.name())
        slowa_klucze_zwrotne.extend(set(synonimy))

        #Hypernims:
        parabole = []
        for par in wordnet.synsets(slowo):
            for p in par.hypernyms():
                for pl in p.lemmas():
                    parabole.append(pl.name())
        slowa_klucze_zwrotne.extend(set(parabole))

        #The others:
        zmiekczenia = []
        for zmi in wordnet.synsets(slowo):
            for zm in zmi.hyponyms():
                for z in zm.lemmas():
                    zmiekczenia.append(z.name())
        slowa_klucze_zwrotne.extend(set(zmiekczenia))

    return(set(slowa_klucze_zwrotne))

In [9]:
#Wyszukiwanie synonimów:
slowa_klucze["perceived_ease_of_use"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["perceived_ease_of_use"])
slowa_klucze["perceived_usefulness"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["perceived_usefulness"] )
slowa_klucze["satisfaction"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["satisfaction"] )
slowa_klucze["attitude"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["attitude"])
slowa_klucze["behavioral_intentions"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["behavioral_intentions"])

In [10]:
#Można wypisać wyniczki - sanity check data presentation:
slowa_klucze["attitude"]

{'A_level',
 'Bos_taurus',
 'CER',
 'GCSE',
 'General_Certificate_of_Secondary_Education',
 'O_level',
 'SPF',
 'ablaut',
 'acceptance',
 'adverse_opinion',
 'affectation',
 'affectedness',
 'aggrade',
 'amplitude_level',
 'anger',
 'angular_unit',
 'anxiety',
 'appraise',
 'asana',
 'assemblage',
 'assess',
 'assort',
 'attitude',
 'authoritative',
 'awe',
 'ballet_position',
 'belief',
 'biosafety_level',
 'bodily_property',
 'caliber',
 'calibre',
 'cattle',
 'centile',
 'choler',
 'class',
 'classify',
 'cognition',
 'college_level',
 'concurring_opinion',
 'conditioned_emotion',
 'conditioned_emotional_response',
 'conjecture',
 'content',
 'course',
 'cows',
 'credence',
 'crucial',
 'culture',
 'decile',
 'decubitus',
 'defensive',
 'defensive_attitude',
 'degree',
 'depth',
 'dictum',
 'discussion_section',
 'disposition',
 'disrespect',
 'dissenting_opinion',
 'downgrade',
 'ectopia',
 'effect',
 'elevation',
 'emotion',
 'emotional_state',
 'esteem',
 'evaluate',
 'evaluation

In [23]:
#Preprocessing danych:
nltk.data.path

['C:\\Users\\admin/nltk_data',
 'C:\\Users\\admin\\anaconda3\\nltk_data',
 'C:\\Users\\admin\\anaconda3\\share\\nltk_data',
 'C:\\Users\\admin\\anaconda3\\lib\\nltk_data',
 'C:\\Users\\admin\\AppData\\Roaming\\nltk_data',
 'C:\\nltk_data',
 'D:\\nltk_data',
 'E:\\nltk_data',
 'data\\lib\\stopwords',
 'data\\lib\\stopwords',
 'data\\lib\\stopwords',
 'data\\lib\\stopwords',
 'data\\lib\\stopwords']

In [24]:
import utils, importlib
importlib.reload(utils) # to keep the .py file up to date when coding
processor = utils.Processor() # Init preprocessing <- there is a path to set up where to dowloads this v
processor.ini_dowload() #dowload all the nessesary files to do the preprocesing (like the dictionary of the stopwords)

[nltk_data] Downloading package stopwords to C:\Users\admin...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\admin...
[nltk_data] Downloading package omw-1.4 to C:\Users\admin...


In [25]:
from nltk import pos_tag
def preprocesing_danych(dane_dane_danedu_dane_danedu_uu):
    dane = dane_dane_danedu_dane_danedu_uu
    dane = processor.preprocess(str(dane))
    dane = processor.tokenize(dane)
    dane = processor.remove_stopwords(dane,remove_len=3) # Remove stopwords (like in the topic task)
    dane = processor.process_tokens(dane)  # Strip the words into the root words
    #dane = pos_tag(dane)
    return dane

In [26]:
slowa_klucze["perceived_ease_of_use"]  = preprocesing_danych(slowa_klucze["perceived_ease_of_use"])
slowa_klucze["perceived_usefulness"]   = preprocesing_danych(slowa_klucze["perceived_usefulness"] )
slowa_klucze["satisfaction"]           = preprocesing_danych(slowa_klucze["satisfaction"]         )
slowa_klucze["attitude"]               = preprocesing_danych(slowa_klucze["attitude"]             )
slowa_klucze["behavioral_intentions"]  = preprocesing_danych(slowa_klucze["behavioral_intentions"])

In [51]:
wszystkie_slowa_klucze = []
wszystkie_slowa_klucze.append(slowa_klucze["perceived_ease_of_use"])
wszystkie_slowa_klucze.append(slowa_klucze["perceived_usefulness"] )
wszystkie_slowa_klucze.append(slowa_klucze["satisfaction"] )
wszystkie_slowa_klucze.append(slowa_klucze["attitude"] )
wszystkie_slowa_klucze.append(slowa_klucze["behavioral_intentions"] )
wszystkie_slowa_klucze

[['angular',
  'veloc',
  'issu',
  'slowli',
  'pickaninni',
  'kiddi',
  'readi',
  'well',
  'easygo',
  'loos',
  'comfort',
  'scoot',
  'chalk',
  'amphetamin',
  'sulphat',
  'comfort',
  'station',
  'airhead',
  'waif',
  'dumbbel',
  'dewi',
  'eye',
  'motion',
  'soul',
  'handi',
  'termin',
  'veloc',
  'brisk',
  'easi',
  'latrin',
  'scamp',
  'stop',
  'number',
  'changel',
  'gadgetri',
  'movement',
  'picaninni',
  'hast',
  'shnook',
  'explain',
  'hasten',
  'shaver',
  'shlepper',
  'forget',
  'person',
  'prosper',
  'acceler',
  'flutter',
  'intuit',
  'execut',
  'speed',
  'wanton',
  'fleet',
  'charg',
  'benni',
  'scud',
  'straightaway',
  'tear',
  'meth',
  'barg',
  'sluttish',
  'poor',
  'fish',
  'person',
  'speed',
  'hasti',
  'pelt',
  'along',
  'babi',
  'simpl',
  'mind',
  'cretin',
  'facil',
  'sprog',
  'progeni',
  'wash',
  'room',
  'dope',
  'rush',
  'scalawag',
  'gradual',
  'leisur',
  'bare',
  'prompt',
  'area',
  'well',

In [52]:
from gensim.corpora import Dictionary
keyword_dictionary = Dictionary(wszystkie_slowa_klucze)
keyword_bow_corpus = [keyword_dictionary.doc2bow(doc) for doc in wszystkie_slowa_klucze] #  dictionary reporting how many words and how many times those words appear
from gensim import corpora, models
keyword_tfidf = models.TfidfModel(keyword_bow_corpus)
corpus_tfidf = keyword_tfidf[keyword_bow_corpus]

In [31]:
#Preproces the reviews:
data["preprocessed_reviews"] = data["content"].apply(lambda x: preprocesing_danych(x))

In [32]:
#Vectorize the reviews:
from gensim.corpora import Dictionary
review_dictionary = Dictionary(data["preprocessed_reviews"])
review_bow_corpus = [review_dictionary.doc2bow(doc) for doc in data["preprocessed_reviews"]] #  dictionary reporting how many words and how many times those words appear
data_vectorized = keyword_tfidf[review_bow_corpus]
data["vectorized_reviews"] = data_vectorized.corpus

In [35]:
#Classification:
from thefuzz import fuzz
from thefuzz import process
import numpy as np
def classify(rewiew,all_keywords,threshold=10):
    rewiew = " ".join(rewiew)
    scores = []
    for keywords in all_keywords:
        keywords = " ".join(keywords)
        scores.append(fuzz.token_set_ratio(rewiew,keywords))
    if max(scores) < threshold: #scores are from 0 to 100
        return -1
    best = np.argmax(scores) #ID of the best keyword
    return best

data["class"] = data["preprocessed_reviews"].apply(lambda x : classify(x, wszystkie_slowa_klucze))

C:\Users\admin\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [36]:
data["class"]

0       4
1       1
2       3
3       1
4       2
       ..
1889    2
1890    2
1891    2
1892    1
1893    2
Name: class, Length: 1894, dtype: int64

In [37]:
# Create five horses of the TAM indicators
horse_ease_of_use     =    data.drop(data[data["class"] != 0].index)
horse_usefulness      =    data.drop(data[data["class"] != 1].index)
horse_satisfaction    =    data.drop(data[data["class"] != 2].index)
horse_attitude        =    data.drop(data[data["class"] != 3].index)
horse_behavioral      =    data.drop(data[data["class"] != 4].index)

In [56]:
#filtering out dataframes
easeofuse = horse_ease_of_use[['score', 'sentiment','at']]
usefulness      =    horse_usefulness[['score', 'sentiment','at']]
satisfaction    =    horse_satisfaction[['score', 'sentiment','at']]
attitude        =    horse_attitude[['score', 'sentiment','at']]
behavioral      =    horse_behavioral[['score', 'sentiment','at']]

In [58]:
from scipy import stats

In [75]:
#finding pvalue and correlation
dfs = [easeofuse,usefulness,satisfaction,attitude,behavioral]
cor = []
pval = []
for i in dfs:
    c,p = stats.pearsonr(i.score,i.sentiment)
    cor.append(c)
    pval.append(p)


In [84]:
correlation = pd.DataFrame(list(zip(cor, pval)),columns =['Correlation', 'pval'])

In [85]:
correlation.index = lista_indykatorow

In [86]:
correlation

,Correlation,pval
perceived_ease_of_use,0.254541,5.187912e-05
perceived_usefulness,0.501945,6.297263e-40
satisfaction,0.466959,2.549909e-25
attitude,0.296308,5.603225e-03
behavioral_intentions,0.242002,6.533143e-04


In [94]:
#Task 7.7
positive_senti = data.drop(data[data["sentiment"] != 1].index)
negative_senti = data.drop(data[data["sentiment"] != -1].index)
positive_senti = positive_senti[['score', 'sentiment','at']]
negative_senti = negative_senti[['score', 'sentiment','at']]

In [97]:
senti = [positive_senti,negative_senti]
cr = []
pv = []
for i in senti:
    c,p = stats.pearsonr(i.score,i.sentiment)
    cr.append(c)
    pv.append(p)

sentiment = pd.DataFrame(list(zip(cr, pv)),columns =['Correlation', 'pval'])
sentiment.index = ["positive sentiment","negative sentiment"]
sentiment

,Correlation,pval
positive sentiment,NaN,NaN
negative sentiment,NaN,NaN
